# Introduction

Provided the data downloaded from www.euets.info as zip file, we extract the data using the *ziploader* sub-module:

# Packages and options

In [10]:
import os
from pyeutl import download_data
from pyeutl.ziploader import (
    get_installations, get_accounts, get_transactions,
    get_compliance, get_account_holders) 

# Get Data

Download the data from euets.info. This has to be done only once:

In [11]:
fn_zip = "./eutl.zip"
if not os.path.exists(fn_zip):
    download_data(fn_out=fn_zip)

Get installation data from zip file. The function merges information from lookup tables into the installation table:

In [12]:
df_inst = get_installations(fn_zip, )
df_inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18103 entries, 0 to 18102
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   18103 non-null  object 
 1   name                 18103 non-null  object 
 2   tradingSystem_id     18103 non-null  object 
 3   registry_id          18103 non-null  object 
 4   activity_id          18103 non-null  int64  
 5   eprtrID              5459 non-null   object 
 6   parentCompany        6390 non-null   object 
 7   subsidiaryCompany    3839 non-null   object 
 8   permitID             16419 non-null  object 
 9   designatorICAO       642 non-null    object 
 10  monitoringID         1655 non-null   object 
 11  monitoringExpiry     0 non-null      float64
 12  monitoringFirstYear  1656 non-null   object 
 13  permitDateExpiry     5300 non-null   object 
 14  isAircraftOperator   18075 non-null  object 
 15  ec748_2009Code       1655 non-null  

Get account holder informations

In [13]:
df_accountHolder = get_account_holders(fn_zip)
df_accountHolder = df_accountHolder.rename(columns = {
    c: "accountHolder" + c.capitalize() for c in df_accountHolder.columns if c!= "id"})
df_accountHolder.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19086 entries, 0 to 19085
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  19086 non-null  int64 
 1   accountHolderTradingsystem_id       19086 non-null  object
 2   accountHolderName                   19060 non-null  object
 3   accountHolderAddressmain            18999 non-null  object
 4   accountHolderPostalcode             18933 non-null  object
 5   accountHolderCity                   19016 non-null  object
 6   accountHolderLegalentityidentifier  2715 non-null   object
 7   accountHolderCountry_id             18974 non-null  object
 8   accountHolderCountry                18974 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.3+ MB


Next, load the account data. The account dataframe includes the information of related look-up tables. If a dataframe with installation or account holder information is passed to the function, this information is also merged into the account table:

In [14]:
df_acc = get_accounts(fn_zip, 
                  df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]],
                    df_accountHolder=df_accountHolder)
df_acc.info()
# export the list with all accounts and related installation and account holder informations
df_acc.to_excel("eutl_account_full_list.xlsx", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44045 entries, 0 to 44044
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  44045 non-null  int64  
 1   tradingSystem_id                    43118 non-null  object 
 2   accountIDESD                        226 non-null    object 
 3   yearValid                           224 non-null    float64
 4   name                                43173 non-null  object 
 5   registry_id                         44045 non-null  object 
 6   accountHolder_id                    43114 non-null  float64
 7   accountType_id                      43091 non-null  object 
 8   isOpen                              43118 non-null  object 
 9   openingDate                         42856 non-null  object 
 10  closingDate                         27066 non-null  object 
 11  commitmentPeriod                    41853

Compliance data follow the same logic:

In [15]:
df_comp = get_compliance(fn_zip, df_installation=df_inst)
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485266 entries, 0 to 485265
Data columns (total 49 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    485266 non-null  object 
 1   installation_id       485266 non-null  object 
 2   year                  485266 non-null  int64  
 3   reportedInSystem_id   485266 non-null  object 
 4   euetsPhase            485042 non-null  object 
 5   compliance_id         221107 non-null  object 
 6   allocatedFree         224648 non-null  float64
 7   allocatedNewEntrance  6897 non-null    float64
 8   allocatedTotal        485266 non-null  int64  
 9   allocated10c          1548 non-null    float64
 10  verified              209188 non-null  float64
 11  verifiedCummulative   209341 non-null  float64
 12  verifiedUpdated       485042 non-null  object 
 13  surrendered           186353 non-null  float64
 14  balance               224 non-null     float64
 15  

Transactions follow the same logic also providing the descriptions provided in the look-up tables. In addition, we can pass a table with account information. The transaction loader also allows to resample the transaction data to a higher frequency: 

In [16]:
freq = "ME"
# get transactions merged with account information
cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
        'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
        'accountCategory', 'installationName', 'activity_id', 'activity', 
        'nace_id', 'nace', "naceCategory" ]
df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                      df_account=df_acc[cols])
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997165 entries, 0 to 1997164
Data columns (total 45 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   date                             datetime64[ns]
 1   id                               int64         
 2   transactionID                    object        
 3   transactionTypeMain_id           int64         
 4   transactionTypeSupplementary_id  int64         
 5   transferringAccount_id           float64       
 6   acquiringAccount_id              float64       
 7   unitType_id                      object        
 8   tradingSystem_id                 object        
 9   acquiringYear                    float64       
 10  transferringYear                 float64       
 11  amount                           int64         
 12  transactionTypeMain              object        
 13  transactionTypeSupplementary     object        
 14  unitType                         o

In [17]:
df_trans

,date,id,transactionID,transactionTypeMain_id,transactionTypeSupplementary_id,transferringAccount_id,acquiringAccount_id,unitType_id,tradingSystem_id,acquiringYear,...,acquiringInstallation_id,acquiringRegistry,acquiringAccountType,acquiringAccountCategory,acquiringInstallationName,acquiringActivity_id,acquiringActivity,acquiringNace_id,acquiringNace,acquiringNaceCategory
0,2005-01-31,1111835,DK1,1,51,201.0,201.0,EUA2005,euets,NaN,...,NaN,Denmark,Party Holding Account,Administrative Account,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-02-28,2992,DK386,10,53,201.0,518.0,EUA2005,euets,NaN,...,DK_222,Denmark,Former Operator Holding Account,Operator Holding Account,Guldborgsund Varme ApS,20.0,Combustion of fuels,35.30,Steam and air conditioning supply,Energy: Steam and air conditioning supply
2,2005-02-28,3846,NL78,10,53,1106.0,1279.0,EUA2005,euets,NaN,...,NaN,Netherlands,Former Operator Holding Account,Operator Holding Account,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-02-28,3847,FI146,10,53,1407.0,1564.0,EUA2005,euets,NaN,...,FI_473,Finland,Former Operator Holding Account,Operator Holding Account,Lämpövoimalaitos,1.0,Combustion installations with a rated thermal ...,35.11,Production of electricity,Energy: Electricity generation
4,2005-02-28,6905,FI277,10,53,1407.0,1714.0,EUA2005,euets,NaN,...,FI_217,Finland,Former Operator Holding Account,Operator Holding Account,Kempeleen keskustan lämpökeskus,99.0,Other activity opted-in pursuant to Article 24...,35.11,Production of electricity,Energy: Electricity generation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997160,2023-03-31,1997011,EU770612,10,17,6409817.0,6409851.0,AEA,esd,NaN,...,NaN,European Commission,EU Effort Sharing Account,None,NaN,NaN,NaN,NaN,NaN,NaN
1997161,2023-03-31,1997012,EU770614,10,17,6409841.0,6409851.0,AEA,esd,NaN,...,NaN,European Commission,EU Effort Sharing Account,None,NaN,NaN,NaN,NaN,NaN,NaN
1997162,2023-03-31,1997013,EU770615,10,17,6409833.0,6409851.0,AEA,esd,NaN,...,NaN,European Commission,EU Effort Sharing Account,None,NaN,NaN,NaN,NaN,NaN,NaN
1997163,2023-03-31,1997014,EU770616,10,17,6409673.0,6409851.0,AEA,esd,NaN,...,NaN,European Commission,EU Effort Sharing Account,None,NaN,NaN,NaN,NaN,NaN,NaN


We can wrap all of this into a single function, e.g.:

In [18]:
def load_data(fn_zip, freq=None):
    """Load installation account and transaction data from zip file
    :param fn_zip: <string> name of zip file with eutl data
    :param freq: <string> frequency for resampling of transaction data"""
    # get installations
    df_inst = get_installations(fn_zip)
    # get accounts with selected installation information
    df_acc = get_accounts(fn_zip, 
                          df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]]) 
    # get compliance data with installation information
    df_comp = get_compliance(fn_zip, df_installation=df_inst)    
    # get transactions merged with account information
    cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
            'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
            'accountCategory', 'installationName', 'activity_id', 'activity', 
            'nace_id', 'nace', "naceCategory" ]
    df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                          df_account=df_acc[cols])
    return df_inst, df_acc, df_comp, df_trans

df_inst, df_acc, df_comp, df_trans = load_data(fn_zip, freq=None)